In [1]:
%load_ext autoreload
%autoreload 2

import logging
from dotenv import load_dotenv
load_dotenv()

logging.basicConfig(level=logging.INFO)

logger = logging.getLogger(__name__)

In [2]:
from projects import set_project_root_for_notebook
set_project_root_for_notebook()

INFO:src.environment:Target: DEV
INFO:src.environment:Developer: FDDIFERD
INFO:src.environment:Schema Name: DEV_FDDIFERD
INFO:src.environment:Use Cache: False
INFO:projects:Current working directory: /Users/donatodiferdinando/Documents/GitHub/ml-snowflake-template


In [3]:
from datetime import datetime
from dateutil.relativedelta import relativedelta
from projects.core.chargebacks_disputes_analysis.main import Service

svc = Service(use_cache=False)
date = (datetime.now() - relativedelta(months=6)).strftime("%Y-%m-%d")
svc.set_df(date)

INFO:snowflake.snowpark:AST state has not been set explicitly. Defaulting to ast_enabled = True.
INFO:projects.core.chargebacks_disputes_analysis.main:Querying data from snowflake
INFO:snowflake.connector.connection:Snowflake Connector for Python Version: 3.18.0, Python Version: 3.10.18, Platform: macOS-26.1-arm64-arm-64bit
INFO:snowflake.connector.connection:Connecting to GLOBAL Snowflake domain
INFO:snowflake.snowpark.session:Snowpark Session information: 
"version" : 1.42.0,
"python.version" : 3.10.18,
"python.connector.version" : 3.18.0,
"python.connector.session.id" : 23301365385810,
"os.name" : Darwin

INFO:src.connection.session:Using database ML_LAYER_DB and schema DEV_FDDIFERD
INFO:src.connection.session:Connected to Snowflake for project CORE
INFO:src.connection.session:  Account: "vx70256-sw09563"
INFO:src.connection.session:  User: "ML_LAYER_USER"
INFO:src.connection.session:  Role: "ML_LAYER_ROLE"
INFO:src.connection.session:  Database: "ML_LAYER_DB"
INFO:src.connection.se

,REF_TXN_ID,TXN_ID,PROCESSOR,TYPE,MERCHANT_ACCOUNT,CARD_NETWORK,BRAND_SLUG,SKU_TYPE_SLUG,CHANNEL_SLUG,TRAFFIC_SOURCE_SHORT_NAME,BIN,OFFER_TYPE,COUNT
0,TXN_1444688129,1.444688e+09,adyen,fraud,InstantCheckmateECOM,visa,InstantCheckmate,Background,Affiliate,FUNM,426684,standard,1
1,TXN_1400169319,1.400169e+09,adyen,fraud,TruthFinderECOM,visa,TruthFinder,Background,Affiliate,821d,425907,standard,1
2,TXN_1384067401,1.384067e+09,adyen,fraud,TruthFinderECOM,visa,TruthFinder,Background,PaidSearch,BING,470793,standard,1
3,TXN_1414729717,1.414730e+09,adyen,fraud,TruthFinderECOM,mastercard,TruthFinder,ReversePhoneLookup,Affiliate,Nuwber,542418,standard,1
4,TXN_1408643275,1.408643e+09,adyen,fraud,TruthFinderECOM,visa,TruthFinder,Background,Organic,OG,406095,standard,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4619168,j1ts04gq,1.451429e+09,braintree,settlement,intelius_instant,visa,Intelius,Background,PaidSearch,PUBREC,403784,standard,1
4619169,1yrpz4hf,NaN,braintree,settlement,Truthfinder_instant_2,None,None,None,None,None,None,None,1
4619170,dseehq8y,NaN,braintree,settlement,Truthfinder_instant_2,None,None,None,None,None,None,None,1
4619171,a7a9m1dy,1.441338e+09,braintree,settlement,intelius_instant,visa,Intelius,ReversePhoneLookup,PushNotifications,PUSHMV,481515,trial,1


In [4]:
from projects.core.chargebacks_disputes_analysis.main import Level

level = [
    Level.PROCESSOR,
    Level.CARD_NETWORK, 
    # Level.BRAND
]

svc.get_ratio_df(level)

INFO:projects.core.chargebacks_disputes_analysis.main:Computing ratio for [<Level.PROCESSOR: 'PROCESSOR'>, <Level.CARD_NETWORK: 'CARD_NETWORK'>]


,PROCESSOR,CARD_NETWORK,FRAUD_DISPUTES_COUNT,SETTLED_COUNT,RATIO,GLOBAL_RATIO,DELTA,DELTA_PERCENTAGE
0,adyen,accel,310,0,inf,0.012514,inf,inf
2,adyen,discover,4,0,inf,0.012514,inf,inf
4,adyen,nyce,332,0,inf,0.012514,inf,inf
5,adyen,pulse,1287,0,inf,0.012514,inf,inf
6,adyen,star,837,0,inf,0.012514,inf,inf
3,adyen,mastercard,15344,281933,0.054424,0.012514,0.041910,4.349034
8,adyen,visadankort,2,66,0.030303,0.012514,0.017789,2.421510
7,adyen,visa,21312,746189,0.028561,0.012514,0.016047,2.282315
13,braintree,mastercard,8811,1044624,0.008435,0.012514,-0.004079,0.674008
12,braintree,discover,702,95153,0.007378,0.012514,-0.005137,0.589542
